# 模块

## 猴子补丁

In [2]:
import importlib
import sys
from collections import defaultdict

_post_import_hooks = defaultdict(list)

class PostImportFinder:
    def __init__(self):
        self._skip = set()

    def find_module(self, fullname, path=None):
        if fullname in self._skip:
            return None
        self._skip.add(fullname)
        return PostImportLoader(self)

class PostImportLoader:
    def __init__(self, finder):
        self._finder = finder

    def load_module(self, fullname):
        importlib.import_module(fullname)
        module = sys.modules[fullname]
        for func in _post_import_hooks[fullname]:
            func(module)
        self._finder._skip.remove(fullname)
        return module

def when_imported(fullname):
    def decorate(func):
        if fullname in sys.modules:
            func(sys.modules[fullname])
        else:
            _post_import_hooks[fullname].append(func)
        return func
    return decorate
    
sys.meta_path.insert(0, PostImportFinder())

In [3]:
@when_imported('threading')
def warn_threads(mod):
    print('Threads? Are you crazy?')

import threading

Threads? Are you crazy?


## 将模块拆分出多个文件

In [8]:
!tree mymodule

mymodule
├── __init__.py
├── a.py
└── b.py

0 directories, 3 files


In [4]:
import mymodule
a = mymodule.A()
a.spam()

b = mymodule.B()
b.bar()

A.spam
B.bar
